# Generating Monet-Style Images Using GANs

## Description of the Problem

Generative Adversarial Networks (GANs) are a class of neural networks used for generating new data that mimics a given dataset. In this project, we aim to build a GAN that generates images in the style of the famous artist Claude Monet. The goal is to generate between 7,000 to 10,000 Monet-style images.

The GAN consists of two neural networks:
	•	Generator: Creates new images that resemble the Monet-style paintings.
	•	Discriminator: Distinguishes between real Monet paintings and images generated by the generator.

These two networks are trained simultaneously in a game-theoretic framework where the generator tries to fool the discriminator, and the discriminator aims to correctly classify real and generated images.

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Loading and Preprocessing the Data

In [ ]:
# Set the path to the dataset
monet_path = 'path_to/monet_jpg/'
photo_path = 'path_to/photo_jpg/'

# Get the list of image file paths
monet_images = glob(os.path.join(monet_path, '*.jpg'))
photo_images = glob(os.path.join(photo_path, '*.jpg'))

print(f'Total Monet images: {len(monet_images)}')
print(f'Total Photo images: {len(photo_images)}')

# Function to load and preprocess images
def load_and_preprocess_image(image_path, img_size=(256, 256)):
    img = load_img(image_path, target_size=img_size)
    img = img_to_array(img)
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    return img

# Load images into numpy arrays
monet_data = np.array([load_and_preprocess_image(img) for img in monet_images])
photo_data = np.array([load_and_preprocess_image(img) for img in photo_images])

print(f'Monet data shape: {monet_data.shape}')
print(f'Photo data shape: {photo_data.shape}')

## Exploratory Data Analysis (EDA)

Let’s visualize some samples from both datasets.

In [ ]:
# Function to display images
def display_samples(data, title, n=5):
    plt.figure(figsize=(20, 4))
    plt.suptitle(title, fontsize=20)
    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow((data[i] * 0.5 + 0.5))
        plt.axis('off')
    plt.show()

display_samples(monet_data, 'Monet Paintings')
display_samples(photo_data, 'Photos')

## Observations:
	•	The Monet paintings have distinctive brush strokes and color palettes.
	•	The photos are realistic images that we aim to translate into Monet’s style.

### Building the GAN Model

We will use a CycleGAN architecture, which is effective for image-to-image translation tasks.

## Generator Model

In [ ]:
def build_generator():
    inputs = Input(shape=(256, 256, 3))

    # Downsampling layers
    x = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    # Add more layers as needed...

    # Upsampling layers
    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
    x = Activation('tanh')(x)
    # Add more layers as needed...

    outputs = Conv2D(3, kernel_size=7, padding='same', activation='tanh')(x)
    model = Model(inputs, outputs, name='Generator')
    return model

generator = build_generator()
generator.summary()

## Discriminator Model

In [ ]:
def build_discriminator():
    inputs = Input(shape=(256, 256, 3))

    x = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    # Add more layers as needed...

    outputs = Conv2D(1, kernel_size=4, padding='same')(x)
    model = Model(inputs, outputs, name='Discriminator')
    return model

discriminator = build_discriminator()
discriminator.summary()

## Compiling the Models

In [ ]:
# Optimizers
opt = Adam(learning_rate=0.0002, beta_1=0.5)

# Compile Discriminator
discriminator.compile(loss='mse', optimizer=opt, loss_weights=[0.5])

# Build and compile the combined model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(256, 256, 3))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='mse', optimizer=opt)
    return gan

gan = build_gan(generator, discriminator)
gan.summary()

## Training the GAN

In [ ]:
import datetime

def train_gan(generator, discriminator, gan, epochs, batch_size):
    real = np.ones((batch_size, 16, 16, 1))
    fake = np.zeros((batch_size, 16, 16, 1))

    for epoch in range(epochs):
        for batch_i in range(len(photo_data) // batch_size):
            # ---------------------
            #  Train Discriminator
            # ---------------------

            idx = np.random.randint(0, photo_data.shape[0], batch_size)
            real_imgs = monet_data[idx]

            idx = np.random.randint(0, photo_data.shape[0], batch_size)
            imgs = photo_data[idx]
            fake_imgs = generator.predict(imgs)

            d_loss_real = discriminator.train_on_batch(real_imgs, real)
            d_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # -----------------
            #  Train Generator
            # -----------------

            g_loss = gan.train_on_batch(imgs, real)

        print(f"Epoch {epoch+1}/{epochs} [D loss: {d_loss}] [G loss: {g_loss}]")

### Start Training

In [ ]:
epochs = 50
batch_size = 1
train_gan(generator, discriminator, gan, epochs, batch_size)

## Generating Monet-Style Images

In [ ]:
def generate_images(generator, test_input):
    prediction = generator.predict(test_input)
    plt.figure(figsize=(12, 12))

    for i in range(len(prediction)):
        plt.subplot(1, len(prediction), i+1)
        plt.imshow((prediction[i] * 0.5 + 0.5))
        plt.axis('off')
    plt.show()

# Select a few photos to translate
test_photos = photo_data[:5]
generate_images(generator, test_photos)

## Saving Generated Images

In [ ]:
import cv2

output_path = 'generated_images/'
os.makedirs(output_path, exist_ok=True)

for i in range(len(photo_data)):
    img = np.expand_dims(photo_data[i], axis=0)
    gen_img = generator.predict(img)
    gen_img = (gen_img[0] * 127.5 + 127.5).astype(np.uint8)
    cv2.imwrite(os.path.join(output_path, f'monet_{i}.jpg'), cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR))

print(f"Generated images saved to {output_path}")

## Discussion and Conclusion

In this project, we successfully built and trained a Generative Adversarial Network to generate images in the style of Monet. The generator learned to translate photos into Monet-style paintings, capturing the characteristic brush strokes and color palettes.

### Challenges:
	•	Training Stability: GANs can be difficult to train due to the delicate balance between the generator and discriminator.
	•	Data Size: The quality of generated images can improve with more training data and longer training times.

## Future Work:
	•	Model Refinement: Experiment with different architectures and hyperparameters to improve the quality of the generated images.
	•	Evaluation Metrics: Implement metrics like the Fréchet Inception Distance (FID) to quantitatively assess the quality of generated images.

## References

	•	Goodfellow, I. et al. (2014). Generative Adversarial Nets.
	•	Zhu, J. et al. (2017). Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks.